In [25]:
from os import listdir
import jsonlines
import sklearn
import itertools
from tqdm import tqdm
from tqdm import trange
from sklearn.preprocessing import MultiLabelBinarizer
import re
import nltk
import pymorphy2 as pm

tknzr = nltk.TweetTokenizer()

In [26]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [19]:
path = listdir('c')
path = sorted(path)
path = path[:4]
print(path)

data = []
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AB', 'AD', 'AE']


In [20]:
categories_y = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories_y))))

categories_dict = {c: 0 for c in categories}
for d in data:
    for c in d['categories']:
        if categories_dict.get(c) is not None:
            categories_dict[c] += 1
            
ok = [c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000]
categories_y = [[c for c in cy if c in ok] for cy in tqdm(categories_y)]
len(ok)

100%|██████████████████████████████████████████████████████████████████████████| 35669/35669 [00:13<00:00, 2581.66it/s]


4781

In [21]:
y = MultiLabelBinarizer().fit_transform(categories_y)

In [24]:
words = []
texts = []
for d in tqdm(data):

    text = re.sub(r'\n+', '. ', d['text'])
    text = re.sub(r'\xa0', ' ', text)
    tokens = tknzr.tokenize(text=text)
    tokens = [t.lower() for t in tokens]     #  if t.isalnum()
    text = ' '.join(tokens)
    texts.append(text)
    words.extend(sorted(set([t for t in tokens if t.isalpha()])))

words = sorted(set(words))

  0%|                                                                                        | 0/35669 [00:00<?, ?it/s]


NameError: name 'tknzr' is not defined

In [ ]:
normal_forms = {}
for w in tqdm(words):
    nf = morph.parse(w)[0].normal_form
    normal_forms[w] = nf

In [ ]:
for i in trange(texts):
    text = texts[i]
    tokens = re.split(r' ', text)
    tokens = [normal_forms.get(t) for t in tokens if normal_forms.get(t) is not None]
    text = ' '.join(tokens)
    texts[i] = text
